In [1]:
import pandas as pd
import csv
from datetime import datetime,date
from sigfig import round as round_sigfig
import json
import subprocess, os
from astropy import units as u
from astropy.coordinates import Angle
import numpy as np

In [2]:
def round_val_err_indiv(val, err, category):
    """
    Rounds the value and error and returns each individual object
    
    Used for creating a csv file rather than a pdf.
    """

    val_rounded, err_rounded = round_sigfig(val, uncertainty=err, cutoff=99, sep=list, type=str)

    return val_rounded, err_rounded

def round_val_err_indiv_DRAKE(val, err, category):
    """
    Rounds the value and error and returns each individual object
    
    Used for creating a csv file rather than a pdf.
    """

    val_err = round_sigfig(val, uncertainty=err, cutoff=99, format = 'Drake', type=str)

    return val_err

In [4]:
data_basecat = pd.read_json('Basecat_morph.json')
data_basecat = data_basecat.fillna(0)

In [5]:
print(len(data_basecat))

137


In [6]:
header = r'''\documentclass{article}
\usepackage{longtable}
\usepackage{gensymb}
\usepackage{graphicx}
\usepackage[a4paper, margin=0.5in]{geometry}
\usepackage{scalerel}
\usepackage{threeparttablex}
\usepackage[T1]{fontenc}
\usepackage[font=normalsize]{caption}
\usepackage{etoolbox}
\appto\TPTnoteSettings{\normalsize}

%\setlength{\LTpre}{0pt}
%\setlength{\LTpost}{0pt}

\graphicspath{{./imgs/}}
\begin{document}
%\begin{landscape}
\begin{center}
'''

footer = r'''\end{center}
%\end{landscape}
\end{document}
'''

# table = ''
# table = table + r'''
# \begin{ThreePartTable} 
# \renewcommand\TPTminimum{\textwidth}
# \begin{longtable}[c]{c c c c c c c}
# \caption{CHIME/FRB First Baseband Catalog}\endfirsthead
# \caption{\textit{continued}} \endhead
# \hline
# \\[-2ex]
# Name \tnote{a} 
# & \begin{tabular}[c]{@{}c@{}}DM\\ (pc cm$^{-3}$)\end{tabular}
# & \begin{tabular}[c]{@{}c@{}}Downsampling\\( )\end{tabular}
# & \begin{tabular}[c]{@{}c@{}}Num Components\\( )\end{tabular}
# & \begin{tabular}[c]{@{}c@{}}Min Width\\(ms)\end{tabular}
# & \begin{tabular}[c]{@{}c@{}}Scattering\\(ms)\end{tabular}
# & \begin{tabular}[c]{@{}c@{}}Bandwidth\\(MHz)\end{tabular}\\
# \\[-2ex]
# \hline
# \\[-2ex]
# '''

table = ''
table = table + r'''
\begin{ThreePartTable} 
\renewcommand\TPTminimum{\textwidth}
\begin{longtable}[c]{c c c c c c c}
\caption{CHIME/FRB First Baseband Catalog Morphology (prelim)}\endfirsthead
\caption{\textit{continued}} \endhead
\hline
\\[-2ex]
TNS Name & DM & Downsample & # Components & Min Width & Scattering & Bandwidth \\
 & (pc cm$^{-3}$) &  &  & (ms) & (ms) & (MHz) \\
\\[-2ex]
\hline
\\[-2ex]
'''

for index, row in data_basecat.iterrows():
    tns_name = row['TNS_name']
    
    type_scat = row['Fit_type']
    
    top_freq = row['Top_avail_freq']
    
    DM = round_sigfig(row['DM'], uncertainty = row['DM_err'], sep='brackets')
    
    tres = int(row['Time_res(s)']/2.56e-6)
    
    ncomp = row['Num_comps']
    
    minw = round_sigfig(row['Min_width(s)']*1e3, uncertainty = row['Min_width_err(s)']*1e3, sep = 'brackets')
    
    if ncomp > 1:
        maxw = round_sigfig(row['Max_width(s)']*1e3, uncertainty = row['Max_width_err(s)']*1e3, sep = 'brackets')
#     else:
#         maxw = 0
    
    
    if type_scat == 'Noscat':
        sc = '$<$ {}'.format(round((row['Scattering(s)']*1e3),2))
    elif type_scat == 'Profile':
        sc = '$\sim$ {}'.format(round((row['Scattering(s)']*1e3),2))
    elif type_scat == 'Failed':
        sc = 'Nan'
    else:
        sc = round_sigfig(row['Scattering(s)']*1e3, uncertainty = row['Scattering_err_sim(s)']*1e3, sep = 'brackets')
    
    if top_freq > 800:
        bw = np.round(row['Bandwidth'],1)
    else:
        bw = '$>$ {}'.format(np.round(row['Bandwidth'],1))
    
    if ncomp > 1:
        line ='{} & {} & {} & {} & {}, {} & {} & {} \\\ '.format(tns_name, DM, tres, ncomp, minw, maxw, sc, bw)
    else:
        line ='{} & {} & {} & {} & {} & {} & {} \\\ '.format(tns_name, DM, tres, ncomp, minw, sc, bw)
    table += line + '\n'
    
    
table = table + r'''\hline
\\[-2ex]
\insertTableNotes
\end{longtable}
\end{ThreePartTable}
\vspace*{1cm}
'''

content = header + table + footer
print(table)
    
    



\begin{ThreePartTable} 
\renewcommand\TPTminimum{\textwidth}
\begin{longtable}[c]{c c c c c c c}
\caption{CHIME/FRB First Baseband Catalog Morphology (prelim)}\endfirsthead
\caption{\textit{continued}} \endhead
\hline
\\[-2ex]
TNS Name & DM & Downsample & # Components & Min Width & Scattering & Bandwidth \\
 & (pc cm$^{-3}$) &  &  & (ms) & (ms) & (MHz) \\
\\[-2ex]
\hline
\\[-2ex]
FRB20181209A & 328.59(1) & 1 & 1 & 0.048(2) & 0.114(4) & 400.0 \\ 
FRB20181213A & 678.69(1) & 8 & 3 & 0.048(6), 0.078(4) & 0.17(5) & 400.0 \\ 
FRB20181214C & 632.827(7) & 32 & 2 & 0.067(6), 0.38(3) & 1.6(2) & 400.0 \\ 
FRB20181215B & 494.044(6) & 1 & 4 & 0.0138(2), 0.196(6) & $\sim$ 0.01 & 400.0 \\ 
FRB20181219C & 647.76(4) & 128 & 1 & 0.61(3) & 1.1(1) & 400.0 \\ 
FRB20181220A & 209.517(8) & 8 & 1 & 0.074(4) & 0.21(1) & 400.0 \\ 
FRB20181221A & 316.29(5) & 64 & 1 & 0.47(2) & 1.38(6) & 133.3 \\ 
FRB20181221B & 1394.84(1) & 2 & 1 & 0.220(4) & 0.275(9) & $>$ 230.7 \\ 
FRB20181222E & 327.949(5) & 32 & 3 & 0.29(1)

In [9]:
for index, row in data_basecat.iterrows():
    print(row['TNS_name'])

FRB20181209A
FRB20181213A
FRB20181214C
FRB20181215B
FRB20181219C
FRB20181220A
FRB20181221A
FRB20181221B
FRB20181222E
FRB20181223C
FRB20181224E
FRB20181225A
FRB20181226A
FRB20181226D
FRB20181226E
FRB20181228B
FRB20181229A
FRB20181231A
FRB20181231B
FRB20181231C
FRB20190102A
FRB20190102B
FRB20190103C
FRB20190106B
FRB20190110A
FRB20190110C
FRB20190111B
FRB20190115B
FRB20190116A
FRB20190117A
FRB20190118A
FRB20190121A
FRB20190122C
FRB20190124B
FRB20190124F
FRB20190130B
FRB20190131E
FRB20190201B
FRB20190202B
FRB20190203A
FRB20190204B
FRB20190206A
FRB20190208C
FRB20190210B
FRB20190212B
FRB20190212C
FRB20190213D
FRB20190214C
FRB20190217A
FRB20190224C
FRB20190224D
FRB20190226A
FRB20190227A
FRB20190301A
FRB20190303B
FRB20190304A
FRB20190304B
FRB20190320A
FRB20190320B
FRB20190320E
FRB20190322A
FRB20190323B
FRB20190327A
FRB20190405B
FRB20190410B
FRB20190411B
FRB20190411C
FRB20190412A
FRB20190417C
FRB20190418A
FRB20190419B
FRB20190420B
FRB20190423A
FRB20190423D
FRB20190425A
FRB20190425B
FRB20190427A